## This notebook shows how to use Orchestrator APIs for user experiments

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
fabric_rc_location=os.environ['HOME']+"/work/fabric_config/fabric_rc"
if os.path.exists(fabric_rc_location):
    with open(fabric_rc_location, 'r') as f:
        for line in f:
            if line.startswith('export'):
                os.environ[line.split('=')[0].split('export')[1].strip()] = line.split('=')[1].strip()

In [ ]:
ssh_key_file_priv=os.environ["FABRIC_SLICE_PRIVATE_KEY_FILE"]
ssh_key_file_pub=os.environ["FABRIC_SLICE_PUBLIC_KEY_FILE"]

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
slice_name="l3vpn"

## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
slice_manager = SliceManager()

### Orchestrator API example to query for available resources

In [ ]:
status, advertised_topology = slice_manager.resources()

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

In [ ]:
if status == Status.OK:
    advertised_topology.draw()

## Create Slice

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType, Labels
from fim.slivers.capacities_labels import Flags

import fim.user as f
t = f.ExperimentTopology()
n1 = t.add_node(name='n1', site='RENC', ntype=f.NodeType.VM)
n2 = t.add_node(name='n2', site='UKY')
n3 = t.add_node(name='n3', site='UKY')

cap = f.Capacities(core=2, ram=8, disk=100)
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')
n2.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')
n3.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')

n1.add_component(model_type=f.ComponentModelType.SmartNIC_ConnectX_6, name='n1-nic1')
n2.add_component(model_type=f.ComponentModelType.SmartNIC_ConnectX_6, name='n2-nic1')
n3.add_component(model_type=f.ComponentModelType.SmartNIC_ConnectX_6, name='n3-nic1')

n1.interface_list[0].labels = Labels.update(n1.interface_list[0].labels, ipv4_subnet='192.168.10.1/24', vlan="1001")
peering_labels = Labels(ipv4_subnet='192.168.10.2/24', asn='654321',  bgp_key='somesecret')
n1.interface_list[0].peer_labels = peering_labels


n2.interface_list[0].labels = Labels.update(n2.interface_list[0].labels, ipv4_subnet='192.168.20.1/24', vlan="1002")
n3.interface_list[0].labels = Labels.update(n3.interface_list[0].labels, ipv4_subnet='192.168.20.1/24', vlan="1003")

ns1 = t.add_network_service(name='ns1', nstype=f.ServiceType.L3VPN,
                      interfaces=[n1.interface_list[0], n2.interface_list[0], n3.interface_list[0]])
        

# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key_pub)


print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
else:
    print(f"Failure: {reservations}")

## Query Slices

In [ ]:
slice_id = reservations[0].slice_id
#slice_id = "1a813e80-82ea-44d1-88b7-151261bcf3b0"
status, slices = slice_manager.slices(slice_id=slice_id)

print("Response Status {}".format(status))
if status == Status.OK:
    slice_object = slices[0]
    print("Slice Id {}".format(slice_object.slice_id))
    print("Slice Name {}".format(slice_object.name))
    print("Slice State {}".format(slice_object.state))
    print("Slice End {}".format(slice_object.lease_end_time))
    #print("Slice End {}".format(slice_object.model))
else:
    print(f"Failure: {slices}")

## Query Slivers

In [ ]:
status, slivers = slice_manager.slivers(slice_object=slice_object)

print("Response Status {}".format(status))
if status == Status.OK:
    for s in slivers:
        print("=====================")
        print("SliceId {}".format(s.slice_id))
        print("ReservationId {}".format(s.sliver_id))
        print("State         {}".format(s.state))
        print("Pending       {}".format(s.pending_state))
        print("Join State    {}".format(s.join_state))
        print("Sliver Type   {}".format(s.sliver_type))
        print("Notice   {}".format(s.notice))
        #print("Sliver        {}".format(s.sliver))
        print("Site        {}".format(s.fim_sliver.site))
        if s.sliver_type == "NodeSliver":
            print("Mgmt IP        {}".format(s.fim_sliver.management_ip))
        else:
            print("Gateway KOMAL       {}".format(s.fim_sliver.gateway))
        
        if s.sliver_type == "NodeSliver":
            for d in s.fim_sliver.attached_components_info.devices.values():
                print("-------------------------")
                print("COMPONENT        {}".format(d))
                print("-------------------------")
        else:
            for d in s.fim_sliver.interface_info.interfaces.values():
                print("-------------------------")
                print("INTERFACE        {}".format(d))
                print("INTERFACE        {}".format(d.flags))
                print("-------------------------")
                
        print("Notices       {}".format(s.notice))
        
        print("=====================")
else:
    print(f"Failure: {slivers}")

## Delete Slice

In [ ]:
status, result = slice_manager.delete(slice_object=slice_object)

print("Response Status {}".format(status))
print("Response received {}".format(result))